Population Estimates, SEDAC	soc.031	http://sedac.ciesin.columbia.edu/data/set/gpw-v4-population-density-adjusted-to-2015-unwpp-country-totals

Import libraries

In [ ]:
# Libraries for downloading data from remote server (may be ftp)
import requests
from urllib.request import urlopen
from contextlib import closing
import shutil

# Library for uploading/downloading data to/from S3
import boto3

# Libraries for handling data
import rasterio as rio
import numpy as np
# from netCDF4 import Dataset
# import pandas as pd
# import scipy

# Libraries for various helper functions
# from datetime import datetime
import os
import threading
import sys
from glob import glob

s3 tools

In [18]:
s3_upload = boto3.client("s3")
s3_download = boto3.resource("s3")

s3_bucket = "wri-public-data"
s3_folder = "resourcewatch/raster/soc_031_population_grid/"

s3_file1 = "soc_031_population_grid_2000.tif"
s3_file2 = "soc_031_population_grid_2005.tif"
s3_file3 = "soc_031_population_grid_2010.tif"
s3_file4 = "soc_031_population_grid_2020.tif"

s3_key_orig1 = s3_folder + s3_file1
s3_key_edit1 = s3_key_orig1[0:-4] + "_edit.tif"

s3_key_orig2 = s3_folder + s3_file2
s3_key_edit2 = s3_key_orig2[0:-4] + "_edit.tif"

s3_key_orig3 = s3_folder + s3_file3
s3_key_edit3 = s3_key_orig3[0:-4] + "_edit.tif"

s3_key_orig4 = s3_folder + s3_file4
s3_key_edit4 = s3_key_orig4[0:-4] + "_edit.tif"

class ProgressPercentage(object):
        def __init__(self, filename):
            self._filename = filename
            self._size = float(os.path.getsize(filename))
            self._seen_so_far = 0
            self._lock = threading.Lock()

        def __call__(self, bytes_amount):
            # To simplify we'll assume this is hooked up
            # to a single filename.
            with self._lock:
                self._seen_so_far += bytes_amount
                percentage = (self._seen_so_far / self._size) * 100
                sys.stdout.write("\r%s  %s / %s  (%.2f%%)"%(
                        self._filename, self._seen_so_far, self._size,
                        percentage))
                sys.stdout.flush()

Potentially useful for online directories

Define local file locations

In [14]:
local_folder = "/Users/Max81007/Desktop/Python/Resource_Watch/Raster/soc_031/"

file_name1 = "gpw-v4-population-density-adjusted-to-2015-unwpp-country-totals_2000.tif"
file_name2 = "gpw-v4-population-density-adjusted-to-2015-unwpp-country-totals_2005.tif"
file_name3 = "gpw-v4-population-density-adjusted-to-2015-unwpp-country-totals_2010.tif"
file_name4 = "gpw-v4-population-density-adjusted-to-2015-unwpp-country-totals_2020.tif"


local_orig1 = local_folder + file_name1
local_orig2 = local_folder + file_name2
local_orig3 = local_folder + file_name3
local_orig4 = local_folder + file_name4

orig_extension_length = 4 #4 for each char in .tif

local_edit1 = local_orig1[:-orig_extension_length] + "edit.tif" 
local_edit2 = local_orig2[:-orig_extension_length] + "edit.tif" 
local_edit3 = local_orig3[:-orig_extension_length] + "edit.tif" 
local_edit4 = local_orig4[:-orig_extension_length] + "edit.tif" 



Use rasterio to reproject and compress

In [15]:
files = [local_orig1, local_orig2, local_orig2, local_orig4]
for file in files:
    with rio.open(file, 'r') as src:
        profile = src.profile
        print(profile)

{'driver': 'GTiff', 'dtype': 'float32', 'nodata': -407649103380480.0, 'width': 43200, 'height': 17400, 'count': 1, 'crs': CRS({'init': 'epsg:4326'}), 'transform': Affine(0.00833333333333339, 0.0, -180.0,
       0.0, -0.00833333333333339, 85.00000000000922), 'blockxsize': 128, 'blockysize': 128, 'tiled': True, 'compress': 'lzw', 'interleave': 'band'}
{'driver': 'GTiff', 'dtype': 'float32', 'nodata': -407649103380480.0, 'width': 43200, 'height': 17400, 'count': 1, 'crs': CRS({'init': 'epsg:4326'}), 'transform': Affine(0.00833333333333339, 0.0, -180.0,
       0.0, -0.00833333333333339, 85.00000000000922), 'blockxsize': 128, 'blockysize': 128, 'tiled': True, 'compress': 'lzw', 'interleave': 'band'}
{'driver': 'GTiff', 'dtype': 'float32', 'nodata': -407649103380480.0, 'width': 43200, 'height': 17400, 'count': 1, 'crs': CRS({'init': 'epsg:4326'}), 'transform': Affine(0.00833333333333339, 0.0, -180.0,
       0.0, -0.00833333333333339, 85.00000000000922), 'blockxsize': 128, 'blockysize': 128, 

In [37]:
# Note - this is the core of Vizz's netcdf2tif function
def reprojection(orig_name, edit_name):
    with rio.open(orig_name, 'r') as src:
        # This assumes data is readable by rasterio
        # May need to open instead with netcdf4.Dataset, for example

        profile = src.profile
        rows = profile["height"]
        columns = profile["width"]

        print(rows)
        print(columns)

        # Latitude bounds
        south_lat = -90
        north_lat = 90

        # Longitude bounds
        west_lon = -180
        east_lon = 180

        transform = rio.transform.from_bounds(west_lon, south_lat, east_lon, north_lat, columns, rows)

        # Profile

        no_data_val = -407649103380480.0
        target_projection = 'EPSG:4326'
        target_data_type = np.float64

        profile = {
            'driver':'GTiff', 
            'height':rows, 
            'width':columns, 
            'count':1, 
            'dtype':target_data_type, 
            'crs':target_projection, 
            'transform':transform, 
            'compress':'lzw', 
            'nodata': no_data_val
        }
        
        windows = src.block_windows()

        with rio.open(edit_name, "w", **profile) as dst:
            for ix, window in windows:
                if(ix[0]%100==0):
                    
                    print(ix)
                data = src.read(1, window=window)
                dst.write(data.astype(profile["dtype"]), 1, window=window)

In [38]:
reprojection(local_orig1, local_edit1)
reprojection(local_orig2, local_edit2)
reprojection(local_orig3, local_edit3)
reprojection(local_orig4, local_edit4)

17400
43200
(0, 0)
(0, 1)
(0, 2)
(0, 3)
(0, 4)
(0, 5)
(0, 6)
(0, 7)
(0, 8)
(0, 9)
(0, 10)
(0, 11)
(0, 12)
(0, 13)
(0, 14)
(0, 15)
(0, 16)
(0, 17)
(0, 18)
(0, 19)
(0, 20)
(0, 21)
(0, 22)
(0, 23)
(0, 24)
(0, 25)
(0, 26)
(0, 27)
(0, 28)
(0, 29)
(0, 30)
(0, 31)
(0, 32)
(0, 33)
(0, 34)
(0, 35)
(0, 36)
(0, 37)
(0, 38)
(0, 39)
(0, 40)
(0, 41)
(0, 42)
(0, 43)
(0, 44)
(0, 45)
(0, 46)
(0, 47)
(0, 48)
(0, 49)
(0, 50)
(0, 51)
(0, 52)
(0, 53)
(0, 54)
(0, 55)
(0, 56)
(0, 57)
(0, 58)
(0, 59)
(0, 60)
(0, 61)
(0, 62)
(0, 63)
(0, 64)
(0, 65)
(0, 66)
(0, 67)
(0, 68)
(0, 69)
(0, 70)
(0, 71)
(0, 72)
(0, 73)
(0, 74)
(0, 75)
(0, 76)
(0, 77)
(0, 78)
(0, 79)
(0, 80)
(0, 81)
(0, 82)
(0, 83)
(0, 84)
(0, 85)
(0, 86)
(0, 87)
(0, 88)
(0, 89)
(0, 90)
(0, 91)
(0, 92)
(0, 93)
(0, 94)
(0, 95)
(0, 96)
(0, 97)
(0, 98)
(0, 99)
(0, 100)
(0, 101)
(0, 102)
(0, 103)
(0, 104)
(0, 105)
(0, 106)
(0, 107)
(0, 108)
(0, 109)
(0, 110)
(0, 111)
(0, 112)
(0, 113)
(0, 114)
(0, 115)
(0, 116)
(0, 117)
(0, 118)
(0, 119)
(0, 120)
(0, 121)


(100, 0)
(100, 1)
(100, 2)
(100, 3)
(100, 4)
(100, 5)
(100, 6)
(100, 7)
(100, 8)
(100, 9)
(100, 10)
(100, 11)
(100, 12)
(100, 13)
(100, 14)
(100, 15)
(100, 16)
(100, 17)
(100, 18)
(100, 19)
(100, 20)
(100, 21)
(100, 22)
(100, 23)
(100, 24)
(100, 25)
(100, 26)
(100, 27)
(100, 28)
(100, 29)
(100, 30)
(100, 31)
(100, 32)
(100, 33)
(100, 34)
(100, 35)
(100, 36)
(100, 37)
(100, 38)
(100, 39)
(100, 40)
(100, 41)
(100, 42)
(100, 43)
(100, 44)
(100, 45)
(100, 46)
(100, 47)
(100, 48)
(100, 49)
(100, 50)
(100, 51)
(100, 52)
(100, 53)
(100, 54)
(100, 55)
(100, 56)
(100, 57)
(100, 58)
(100, 59)
(100, 60)
(100, 61)
(100, 62)
(100, 63)
(100, 64)
(100, 65)
(100, 66)
(100, 67)
(100, 68)
(100, 69)
(100, 70)
(100, 71)
(100, 72)
(100, 73)
(100, 74)
(100, 75)
(100, 76)
(100, 77)
(100, 78)
(100, 79)
(100, 80)
(100, 81)
(100, 82)
(100, 83)
(100, 84)
(100, 85)
(100, 86)
(100, 87)
(100, 88)
(100, 89)
(100, 90)
(100, 91)
(100, 92)
(100, 93)
(100, 94)
(100, 95)
(100, 96)
(100, 97)
(100, 98)
(100, 99)
(100, 100)

(100, 325)
(100, 326)
(100, 327)
(100, 328)
(100, 329)
(100, 330)
(100, 331)
(100, 332)
(100, 333)
(100, 334)
(100, 335)
(100, 336)
(100, 337)
17400
43200
(0, 0)
(0, 1)
(0, 2)
(0, 3)
(0, 4)
(0, 5)
(0, 6)
(0, 7)
(0, 8)
(0, 9)
(0, 10)
(0, 11)
(0, 12)
(0, 13)
(0, 14)
(0, 15)
(0, 16)
(0, 17)
(0, 18)
(0, 19)
(0, 20)
(0, 21)
(0, 22)
(0, 23)
(0, 24)
(0, 25)
(0, 26)
(0, 27)
(0, 28)
(0, 29)
(0, 30)
(0, 31)
(0, 32)
(0, 33)
(0, 34)
(0, 35)
(0, 36)
(0, 37)
(0, 38)
(0, 39)
(0, 40)
(0, 41)
(0, 42)
(0, 43)
(0, 44)
(0, 45)
(0, 46)
(0, 47)
(0, 48)
(0, 49)
(0, 50)
(0, 51)
(0, 52)
(0, 53)
(0, 54)
(0, 55)
(0, 56)
(0, 57)
(0, 58)
(0, 59)
(0, 60)
(0, 61)
(0, 62)
(0, 63)
(0, 64)
(0, 65)
(0, 66)
(0, 67)
(0, 68)
(0, 69)
(0, 70)
(0, 71)
(0, 72)
(0, 73)
(0, 74)
(0, 75)
(0, 76)
(0, 77)
(0, 78)
(0, 79)
(0, 80)
(0, 81)
(0, 82)
(0, 83)
(0, 84)
(0, 85)
(0, 86)
(0, 87)
(0, 88)
(0, 89)
(0, 90)
(0, 91)
(0, 92)
(0, 93)
(0, 94)
(0, 95)
(0, 96)
(0, 97)
(0, 98)
(0, 99)
(0, 100)
(0, 101)
(0, 102)
(0, 103)
(0, 104)
(0, 105)
(

Upload orig and edit files to s3

In [39]:
# Original
s3_upload.upload_file(local_orig1, s3_bucket, s3_key_orig1,
                         Callback=ProgressPercentage(local_orig1))
s3_upload.upload_file(local_orig2, s3_bucket, s3_key_orig2,
                         Callback=ProgressPercentage(local_orig2))
s3_upload.upload_file(local_orig3, s3_bucket, s3_key_orig3,
                         Callback=ProgressPercentage(local_orig3))
s3_upload.upload_file(local_orig4, s3_bucket, s3_key_orig4,
                         Callback=ProgressPercentage(local_orig4))

# Edit
s3_upload.upload_file(local_edit1, s3_bucket, s3_key_edit1,
                         Callback=ProgressPercentage(local_edit1))
s3_upload.upload_file(local_edit2, s3_bucket, s3_key_edit2,
                         Callback=ProgressPercentage(local_edit2))
s3_upload.upload_file(local_edit3, s3_bucket, s3_key_edit3,
                         Callback=ProgressPercentage(local_edit3))
s3_upload.upload_file(local_edit4, s3_bucket, s3_key_edit4,
                         Callback=ProgressPercentage(local_edit4))

/Users/Max81007/Desktop/Python/Resource_Watch/Raster/soc_031/gpw-v4-population-density-adjusted-to-2015-unwpp-country-totals_2020edit.tif  483771574 / 483771574.0  (100.00%)

In [29]:
with rio.open(local_orig1, 'r') as src:
    print(src.profile)
    windows = src.block_windows()
    
    for win_tuple in windows:
        print(win_tuple)
        break
    
    for ix, window in windows:
        print(ix)
        print(window)
        data = src.read(window=window)
        print(data)
        break

{'driver': 'GTiff', 'dtype': 'float32', 'nodata': -407649103380480.0, 'width': 43200, 'height': 17400, 'count': 1, 'crs': CRS({'init': 'epsg:4326'}), 'transform': Affine(0.00833333333333339, 0.0, -180.0,
       0.0, -0.00833333333333339, 85.00000000000922), 'blockxsize': 128, 'blockysize': 128, 'tiled': True, 'compress': 'lzw', 'interleave': 'band'}
((0, 0), ((0, 128), (0, 128)))
(0, 1)
((0, 128), (128, 256))
[[[ -4.07649103e+14  -4.07649103e+14  -4.07649103e+14 ...,  -4.07649103e+14
    -4.07649103e+14  -4.07649103e+14]
  [ -4.07649103e+14  -4.07649103e+14  -4.07649103e+14 ...,  -4.07649103e+14
    -4.07649103e+14  -4.07649103e+14]
  [ -4.07649103e+14  -4.07649103e+14  -4.07649103e+14 ...,  -4.07649103e+14
    -4.07649103e+14  -4.07649103e+14]
  ..., 
  [ -4.07649103e+14  -4.07649103e+14  -4.07649103e+14 ...,  -4.07649103e+14
    -4.07649103e+14  -4.07649103e+14]
  [ -4.07649103e+14  -4.07649103e+14  -4.07649103e+14 ...,  -4.07649103e+14
    -4.07649103e+14  -4.07649103e+14]
  [ -4.07

In [40]:
import arcpy 

ModuleNotFoundError: No module named 'arcpy'